In [1]:
# Primary modules
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# Other modules / functions
import pickle
import os
import matplotlib.gridspec as gridspec

# For inline plotting
%matplotlib inline
plt.ion()

In [2]:
# Import suftware (use development copy)
import sys
sys.path.append('../') #('../../suftware')

import suftware as sw
from supplemental_functions import gkde, dpmm, KL_divergence, p_value_cal

In [3]:
# Specify simulation parameters
num_simulations = 100
N = 100

# Specify DEFT parameters
num_samples_for_Z = 100000
num_posterior_samples = 100

# Save result ?
save_result = True
dir_result = './simulations_test/figS2/'
col_names = ['Feynman Diagrams', 'Importance Sampling', 'Importance Sampling errorbar']

In [4]:
distribution_names = ['DoubleGaussianZoom', 'DoubleGaussian']
    
for i, distribution_name in enumerate(distribution_names):
    print('Simulating %s...'%distribution_name)
    print()
    
    dist = sw.SimulatedDensity(name=distribution_name)
    xs = np.linspace(dist.xmin, dist.xmax, 100)
    
    #
    # Start iteration
    #
    
    log_Z_corrections = np.zeros([num_simulations,3]) 

    i = 0
    while i < num_simulations:

        #
        # Generate data
        #
        
        zs = np.linspace(dist.xmin, dist.xmax, 10001)
        dz = zs[1] - zs[0]
        zs = np.linspace(dist.xmin+dz/2, dist.xmax-dz/2, 10000)
        probs = dist.evaluate(zs) / np.sum(dist.evaluate(zs))
        data = np.random.choice(zs, size=N, replace=True, p=probs)
        jitter = dz * (np.random.rand(N)-0.5)
        data += jitter

        #
        # DEFT with Feynman diagrams
        #

        print('Lap+Fey ...')
        density = sw.DensityEstimator(data, grid=xs, evaluation_method_for_Z='Lap+Fey')        
        points = density.results.points
        ibest = density.results.i_star
        log_Z_corrections[i,0] = np.array([p.log_Z_correction for p in points])[ibest]

        #
        # DEFT with importance sampling 
        #

        print('Lap+Imp ...')
        density = sw.DensityEstimator(data, grid=xs, evaluation_method_for_Z='Lap+Imp', 
                                      num_samples_for_Z=num_samples_for_Z)
        points = density.results.points
        ibest = density.results.i_star
        log_Z_corrections[i,1] = np.array([p.log_Z_correction for p in points])[ibest]
        
        # Compute errorbar of log_Z_correction from importance sampling
        sample_mean_star = np.array([p.sample_mean for p in points])[ibest]
        sample_mean_std_dev_star = np.array([p.sample_mean_std_dev for p in points])[ibest]
        log_Z_corrections[i,2] = sample_mean_std_dev_star / sample_mean_star
            
        #
        # Proceed
        #
        
        print('simulation   # %s' % i)
        print()
        i += 1
        
    #
    # Draw posterior samples using the last dataset
    #
    
    density = sw.DensityEstimator(data, grid=xs, num_posterior_samples=num_posterior_samples)
    Q_samples = density.evaluate_samples(xs, resample=False)
    
    #
    # Save result
    #
    
    if save_result:
        if distribution_name == 'DoubleGaussianZoom':
            bbox_size = 'small'
        elif distribution_name == 'DoubleGaussian':
            bbox_size = 'large'
        
        # Save log_Z_corrections
        pd.DataFrame(data=log_Z_corrections, columns=col_names) \
            .to_pickle(dir_result+'df_log_Z_corrections_bbox_'+bbox_size+'.pkl')
            
        # Save posterior_samples
        pd.DataFrame(data=Q_samples) \
            .to_pickle(dir_result+'df_posterior_samples_bbox_'+bbox_size+'.pkl')
    
print('--- Done ---')
print()

Simulating DoubleGaussianZoom...

Lap+Fey ...
Lap+Imp ...
simulation   # 0

Lap+Fey ...
Lap+Imp ...
simulation   # 1

Lap+Fey ...
Lap+Imp ...
simulation   # 2

Lap+Fey ...
Lap+Imp ...
simulation   # 3

Lap+Fey ...
Lap+Imp ...
simulation   # 4

Lap+Fey ...
Lap+Imp ...
simulation   # 5

Lap+Fey ...
Lap+Imp ...
simulation   # 6

Lap+Fey ...
Lap+Imp ...
simulation   # 7

Lap+Fey ...
Lap+Imp ...
simulation   # 8

Lap+Fey ...
Lap+Imp ...
simulation   # 9

Lap+Fey ...
Lap+Imp ...
simulation   # 10

Lap+Fey ...
Lap+Imp ...
simulation   # 11

Lap+Fey ...
Lap+Imp ...
simulation   # 12

Lap+Fey ...
Lap+Imp ...
simulation   # 13

Lap+Fey ...
Lap+Imp ...
simulation   # 14

Lap+Fey ...
Lap+Imp ...
simulation   # 15

Lap+Fey ...
Lap+Imp ...
simulation   # 16

Lap+Fey ...
Lap+Imp ...
simulation   # 17

Lap+Fey ...
Lap+Imp ...
simulation   # 18

Lap+Fey ...
Lap+Imp ...
simulation   # 19

Lap+Fey ...
Lap+Imp ...
simulation   # 20

Lap+Fey ...
Lap+Imp ...
simulation   # 21

Lap+Fey ...
Lap+Imp ...
simula

simulation   # 89

Lap+Fey ...
Lap+Imp ...
simulation   # 90

Lap+Fey ...
Lap+Imp ...
simulation   # 91

Lap+Fey ...
Lap+Imp ...
simulation   # 92

Lap+Fey ...
Lap+Imp ...
simulation   # 93

Lap+Fey ...
Lap+Imp ...
simulation   # 94

Lap+Fey ...
Lap+Imp ...
simulation   # 95

Lap+Fey ...
Lap+Imp ...
simulation   # 96

Lap+Fey ...
Lap+Imp ...
simulation   # 97

Lap+Fey ...
Lap+Imp ...
simulation   # 98

Lap+Fey ...
Lap+Imp ...
simulation   # 99

--- Done ---

